In [ ]:
!pip install -r requirements.txt


# below fixes some bugs introduced by some recent Colab changes
!mkdir -p /usr/share/vulkan/icd.d
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill2/main/docker/nvidia_icd.json
!wget -q https://raw.githubusercontent.com/haosulab/ManiSkill2/main/docker/10_nvidia.json
!mv nvidia_icd.json /usr/share/vulkan/icd.d
!mv 10_nvidia.json /usr/share/glvnd/egl_vendor.d/10_nvidia.json
# dependencies
#!pip install setuptools==65.5.0
!apt-get install -y --no-install-recommends libvulkan-dev
!pip install mani_skill2
!pip install --upgrade --no-cache-dir gdown

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import gymnasium as gym
import mani_skill2.envs
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
import wandb
wandb.login()

# from collections import OrderedDict
# import torch

# import matplotlib.pyplot as plt
# from google.colab import drive


# import parameters as params
# from utils import save_checkpoint, load_checkpoint

In [ ]:
env_id = "PickCube-v0"
obs_mode = "rgbd"
control_mode = "pd_joint_delta_pos"
reward_mode = "dense"

env = gym.make(env_id,
               obs_mode=obs_mode,
               reward_mode=reward_mode,
               control_mode=control_mode,
               enable_shadow=False)
obs, _ = env.reset()
print("Action Space:", env.action_space)

# take a look at the current state
img = env.unwrapped.render_cameras()
plt.figure(figsize=(10,6))
plt.title("Current State viewed through all RGB and Depth cameras")
plt.imshow(img)

In [ ]:
dataset = ManiSkill2Dataset(f"demos/v0/rigid_body/{env_id}/trajectory.state.pd_ee_delta_pose.h5")
dataloader = DataLoader(dataset, batch_size=256, num_workers=0, pin_memory=True, drop_last=False, shuffle=False)
obs, action = dataset[0]
print("Observation:", obs.shape)
print("Action:", action.shape)

In [ ]:
from train import AgentTrainer
from utils import convert_demonstration

agent_trainer = AgentTrainer() 

""" Pre-train the policy using the demonstrations """
num_episode = 70000
best_epoch_loss = np.inf
pbar = tqdm(dataloader, total=num_episode)
epoch = 0
steps = 0

for steps in range(num_episode):
    epoch_loss = 0
    for batch in dataloader:
        steps += 1
        goal, current, actions, video, proprioception = convert_demonstration(batch)
        loss_val = agent_trainer.pre_train(goal, current, actions, video, proprioception)

        # track the loss and print it
        metrics = {"train/train_loss": loss_val}
        wandb.log(metrics)

        epoch_loss += loss_val
        pbar.set_postfix(dict(loss=loss_val))
        pbar.update(1)

        # periodically save the policy
        if steps % 10000 == 0: agent_trainer.save_checkpoint('check_point')
        if steps >= num_episode: break

    epoch_loss = epoch_loss / len(dataloader)

    # save a new model if the average loss in an epoch has improved
    if epoch_loss < best_epoch_loss:
        best_epoch_loss = epoch_loss
        agent_trainer.save_model("check_point")

    metrics = {"epoch": epoch,
               "train/loss_epoch": epoch_loss}
    wandb.log(metrics)
    epoch += 1

agent_trainer.save_checkpoint('check_point')





# class SerialModelTrain():
#   def __init__(self):
#     self.num_episode = 10
#     self.initial_memory_size = 5
#     self.episode_rewards = []
#     self.num_average_epidodes = 100
#     self.save_every = 100

#     self.max_steps = 100
#     self.envs = Stack('Panda')
#     self.agent_trainer = AgentTrainer()

#     self.evaluate_interval = 10
#     self.reward_window = 20
#     self.reward_values = [0]*self.reward_window
#     self.averge_reward = 0

#   def init_buffer(self):
#     """ Initially, put the data into the replay buffer when an action with noise was taken """
#     state = OrderedDict()
#     next_state = OrderedDict()
#     done = False
#     reward = 0

#     state = self.env.reset()

#     for step in range(self.initial_memory_size):
#         if step % self.max_steps == 0:
#           state = self.env.reset()

#         action = np.random.randn(params.action_dim) # sample random action
#         next_state, reward, done, info = self.env.step(action)

#         # verify the following in the server
#         vision = image_process(state['frontview_image'])
#         vision_next = image_process(next_state['frontview_image'])
#         action = torch.tensor(action, dtype=torch.float32)
#         action = action.squeeze(0)

#         self.agent_trainer.pri_buffer.store(state, action, reward, next_state, done)
#         state = next_state

#     print('%d Data collected' % self.initial_memory_size)


#   def train_model(self):
#     state = OrderedDict()
#     next_state = OrderedDict()
#     done = False
#     reward = 0

#     for episode in range(self.num_episode):
#       state = self.env.reset()
#       episode_reward = 0
#       for t in range(self.max_steps):
#         vision = image_process(state['frontview_image'])

#         robot_state = [
#             np.array(state['robot0_eef_pos'], dtype=np.float32).flatten(),
#             np.array(state['robot0_eef_quat'], dtype=np.float32).flatten()
#         ]

#         robot_state = np.concatenate(robot_state)
#         robot_state = torch.tensor(robot_state, dtype=torch.float32)

#         action = self.agent_trainer.get_action(vision, robot_state)
#         next_state, reward, done, info = self.env.step(action[0])

#         if len(self.reward_value)>=20:
#           self.reward_value.pop(0)
#           self.reward_value.append(reward)

#         # verify the following in the server
#         vision_next = image_process(next_state['frontview_image'])
#         vision = vision.squeeze(0)
#         vision_next = vision_next.squeeze(0)
#         action = torch.tensor(action, dtype=torch.float32)
#         action = action.squeeze(0)

#         self.agent_trainer.pri_buffer.store(state, action, reward, next_state, done)
#         state = next_state
#         if any(done):
#           break

#       episode_reward += reward

#       if episode % 5 == 0:
#         self.agent_trainer.update()
#       self.episode_rewards.append(episode_reward)
#       if episode % self.evaluate_interval == 0:
#         self.averge_reward = np.mean(self.reward_value)
#         print(self.averge_reward)
#       if episode % 100 == 0:
#         print("Episode %d finished | Episode reward %f" % (episode, episode_reward))
#       if episode % self.save_every == 0:
#         self.agent_trainer.save_checkpoint(episode)

#     self.env.close()

#   def plot(self):
#     # Compute the moving average of cumulative rewards
#     moving_average = np.convolve(self.episode_rewards, np.ones(self.num_average_epidodes)/self.num_average_epidodes, mode='valid')
#     plt.plot(np.arange(len(moving_average)),moving_average)
#     plt.title('Average rewards in %d episodes' % self.num_average_epidodes)
#     plt.xlabel('episode')
#     plt.ylabel('rewards')
#     plt.show()

#   def test_model(self):
#     env = Stack('Panda')
#     states = env.reset()
#     self.agent_trainer.load_checkpoint('/content/drive/My Drive/check_point')

#     frames = []
#     for i in range(100):
#       frames.append(states['frontview_image'])  # Append the image to the frames list
#       obs = image_process(states['frontview_image'])
#       action = self.agent_trainer.get_action(obs)

#       next_state, reward, done, info = env.step(action[0])
#       state = next_state


#     imageio.mimwrite('robosuite_video.mp4', frames, fps=20)

In [ ]:
init_memory_size = 1000
observation, _ = env.reset()
num_episode = 10000
max_steps = 100
save_every = 100
episode_rewards = []

for _ in range(init_memory_size):

    vision = observation["image"]["base_camera"]["rgb"] / 255.0
    proprioception = observation["state"]

    action = agent_trainer.get_action(goal, vision, proprioception)
    next_observation, reward, done, truncated, info = env.step(action)

    next_vision = next_observation["image"]["base_camera"]["rgb"] / 255.0
    next_proprioception = next_observation["state"]
    agent_trainer.pri_buffer.store(goal, reward, vision, next_vision, next_proprioception, done)

    observation = next_observation

    if done:
        observation, _ = env.reset()


for epoch in range(num_episode):
    observation, _ = env.reset()
    episode_reward = 0
    for t in range(max_steps):

        vision = observation["image"]["base_camera"]["rgb"] / 255.0
        proprioception = observation["state"]
        action = agent_trainer.get_action(goal, vision, proprioception)
        next_observation, reward, done, truncated, info = env.step(action)

        agent_trainer.pri_buffer.store(goal, reward, vision, next_vision, next_proprioception, done)

        observation = next_observation
        if done:
            break

    episode_reward += reward

    if epoch % 5 == 0:
        agent_trainer.fine_tune()

    episode_rewards.append(episode_reward)

    if epoch % 100 == 0:
        metrics = {"train/episode": epoch, 
                   "train/episode_reward": episode_reward}
        wandb.log(metrics)
    if epoch % save_every == 0:
        agent_trainer.save_checkpoint("check_point")


env.close()

## Train agent

In [ ]:
# # drive.mount('/content/drive')

# serial_train = SerialModelTrain()
# serial_train.init_buffer()
# serial_train.train_model()
# serial_train.plot()


## Test Model

In [ ]:
# serial_train.test_model()

In [ ]:
# """ Load video and encode it in base64 format """
# video_path = 'robosuite_video.mp4'
# video_data = open(video_path, 'rb').read()
# video_encoded = b64encode(video_data).decode()

# # Display video using HTML
# HTML(f"""
# <video width="640" height="480" controls>
#   <source src="data:video/mp4;base64,{video_encoded}" type="video/mp4">
# </video>
# """)